# Library

In [1]:
import pandas as pd
import os
import math
from tqdm import tqdm
from datetime import datetime
import numpy as np

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold

In [3]:
raw1_fold = 'D:/data/big2/BD1/raw'
an3_fold = 'D:/data/big2/BD3/an'
ods_fold = 'D:/data/big2/ODS'
big2_fold = 'D:/data/big2'

# 차량 폐차 현황

## 1\. [로드] 데이터

### 1-1\. 등록&제원(result 4등급)

In [12]:
# about 11.1s
name = 'result(전체)_CAPSS_배출량(Grade추가)(dong_code추가)(2023.05.18_AM.09.49.29)'
file_name = f'{name}.csv'
smig = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False, nrows=5)
smig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   차대번호           5 non-null      object 
 1   차량번호           5 non-null      object 
 2   제원관리번호         5 non-null      int64  
 3   차종             5 non-null      object 
 4   용도             5 non-null      object 
 5   최초등록일          5 non-null      int64  
 6   연식             5 non-null      int64  
 7   제작일자           5 non-null      float64
 8   검사유효일          5 non-null      float64
 9   배출가스인증번호       5 non-null      object 
 10  등급             5 non-null      int64  
 11  차명             5 non-null      object 
 12  차종분류           5 non-null      object 
 13  차종유형           5 non-null      object 
 14  자동차형식          5 non-null      object 
 15  제작사명           5 non-null      object 
 16  연료             5 non-null      object 
 17  엔진형식           5 non-null      object 
 18  총중량           

In [8]:
smig['등급_수정'].value_counts(dropna=False)

등급_수정
4.00    1497678
Name: count, dtype: int64

In [10]:
smig.shape, len(smig['차량관리번호'].unique())

((29534047, 12), 29534043)

In [11]:
len(smig['차대번호'].unique())

29521505

### 1-2\. CEG_CAR_HISTORY_MIG_230420_수정01
- 변경구분코드 : CEGO, CEGC 제외

In [12]:
# about 2m 25.8s
name = 'CEG_CAR_HISTORY_MIG_230420_수정01'
file_name = f'{name}.csv'
his = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36267031 entries, 0 to 36267030
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   차량관리번호  object 
 1   차대번호    object 
 2   차량말소YN  object 
 3   변경일자    float64
 4   변경구분코드  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 1.4+ GB


In [13]:
his.shape, len(his['차량관리번호'].unique())

In [ ]:
len(his['차대번호'].unique())

19400662

### 1-3\. 법정동코드

In [ ]:
# about 8.9s
name = '법정동코드_2022.10.14(수정2023.05.04)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(big2_fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46342 non-null  int64  
 1   시도        46342 non-null  object 
 2   시구군       46319 non-null  object 
 3   읍면동       45849 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46342 non-null  object 
 7   대기관리권역    46342 non-null  object 
 8   대기관리권역YN  46337 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


## 2\. 차량관리번호 기준으로 병합

In [17]:
# about 2m 0.3s
df = smig.merge(his, on='차량관리번호', how='left')
df.shape

(46401826, 16)

In [18]:
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호_x', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_x', '차명', '연료', '등급', '차대번호_y', '차량말소YN_y', '변경일자',
       '변경구분코드'],
      dtype='object')

In [19]:
df = df.drop('차대번호_y', axis=1)
df = df.rename(columns={'차량말소YN_x':'차량말소YN_등록정보', '차량말소YN_y':'차량말소YN_변경이력', '차대번호_x':'차대번호'})
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력', '변경일자',
       '변경구분코드'],
      dtype='object')

## 3\. 차량말소(등록과 변경이력) : Y 추출

In [20]:
df = df.loc[(df['차량말소YN_등록정보'] == 'Y') & (df['차량말소YN_변경이력'] == 'Y')].reset_index(drop=True)
df.shape

(3671025, 15)

In [21]:
df['변경일자'].isnull().sum()

0

In [22]:
df['변경일자'].describe()

count    3671025.00
mean    20204821.91
std         9403.21
min     19971111.00
25%     20200306.00
50%     20201130.00
75%     20210818.00
max     20220628.00
Name: 변경일자, dtype: float64

## 4\. 차대번호 중복 제거(최신 변경일자 기준)

In [23]:
# about 13.5s
df = df.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
df.shape

(3642360, 15)

## 5\. 변경일자 2019.01.01 이후만 추출

In [24]:
df.loc[df['변경일자'] >= 20190101].shape

(3642251, 15)

In [25]:
# about 1.4s
df = df.loc[df['변경일자'] >= 20190101].reset_index(drop=True)
df.shape

(3642251, 15)

In [26]:
df['변경일자'].describe()

count    3642251.00
mean    20204829.13
std         9393.15
min     20190108.00
25%     20200306.00
50%     20201130.00
75%     20210818.00
max     20220628.00
Name: 변경일자, dtype: float64

## 6\. 법정동 코드 병합

In [27]:
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력', '변경일자',
       '변경구분코드'],
      dtype='object')

In [28]:
code.columns

Index(['법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'], dtype='object')

In [29]:
df.shape

(3642251, 15)

In [30]:
df = df.merge(code, left_on='본거지법정동코드', right_on='법정동코드', how='left')
df.shape

(3642251, 24)

In [31]:
df['시도'].isnull().sum()

0

### [출력] 전처리 완료 파일

In [32]:
today_date = datetime.today().strftime("%Y.%m.%d")
today_date

'2023.05.24'

In [33]:
# # about 33.3s
# df.to_csv(os.path.join(ods_fold, f'말소정보_전처리완료({today_date}).csv'), index=False)

# [로드] 전처리 완료 파일

In [7]:
# # about 5m 3.9s
# name = '말소정보_전처리완료(2023.05.24)'
# file_name = f'{name}.csv'
# df = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642251 entries, 0 to 3642250
Data columns (total 24 columns):
 #   Column       Dtype  
---  ------       -----  
 0   차량관리번호       object 
 1   본거지법정동코드     int64  
 2   차종_등록정보      object 
 3   차대번호         object 
 4   차량연식         int64  
 5   제원관리번호       object 
 6   배출가스인증번호     object 
 7   배출가스등급       object 
 8   차량말소YN_등록정보  object 
 9   차명           object 
 10  연료           object 
 11  등급           object 
 12  차량말소YN_변경이력  object 
 13  변경일자         float64
 14  변경구분코드       float64
 15  법정동코드        int64  
 16  시도           object 
 17  시구군          object 
 18  읍면동          object 
 19  리            object 
 20  지역3          float64
 21  폐지여부         object 
 22  대기관리권역       object 
 23  대기관리권역YN     object 
dtypes: float64(3), int64(3), object(18)
memory usage: 666.9+ MB


## 7\. 월별 폐차 현황

In [9]:
df.shape

(3642251, 24)

In [10]:
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력', '변경일자',
       '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역',
       '대기관리권역YN'],
      dtype='object')

In [11]:
'20190101'[:4], '20190101'[4:6], '20190101'[6:]

('2019', '01', '01')

In [12]:
df['변경일자'] = df['변경일자'].astype('str')

In [13]:
df['변경일자_년'] = df['변경일자'].str[:4]
df['변경일자_월'] = df['변경일자'].str[4:6]
df['변경일자_일'] = df['변경일자'].str[6:8]

In [14]:
df['변경일자_년_월'] = df['변경일자_년'] + '.' + df['변경일자_월']

In [15]:
df[['변경일자_년', '변경일자_월', '변경일자_일', '변경일자_년_월']].head()

,변경일자_년,변경일자_월,변경일자_일,변경일자_년_월
0,2022,06,28,2022.06
1,2022,06,28,2022.06
2,2022,06,28,2022.06
3,2022,06,28,2022.06
4,2022,06,28,2022.06


In [16]:
df[['변경일자_년', '변경일자_월', '변경일자_일']] = df[['변경일자_년', '변경일자_월', '변경일자_일']].astype('int')

In [17]:
df['변경일자'] = df['변경일자_년'] + df['변경일자_월'] + df['변경일자_일']
df['변경일자'] = df['변경일자'].astype('int')

In [18]:
df['변경일자'].isnull().sum()

0

In [19]:
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력', '변경일자',
       '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역',
       '대기관리권역YN', '변경일자_년', '변경일자_월', '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [20]:
df['등급'].unique()

array(['X', '2.0', '5.0', '3.0', '4.0', '1.0', nan], dtype=object)

## 4등급 차량에 EG 정보 추가

In [22]:
smig.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력', '변경일자',
       '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역',
       '대기관리권역YN', '변경일자_년', '변경일자_월', '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [23]:
df.columns

Index(['차대번호', '차종유형', 'Grade'], dtype='object')

In [ ]:
dg4 = smig.merge(df, on='차대번호', how='left')
dg4.shape

In [29]:
dg4['Grade'].isnull().sum()

366924

In [ ]:
chc_col = {
    '차량관리번호',
    '본거지법정동코드', 
    '차종_등록정보', 
    '차대번호', '차량연식', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '차량말소YN_등록정보', '차명', '연료', '등급', '차량말소YN_변경이력', '변경일자',
       '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역',
       '대기관리권역YN', '변경일자_년', '변경일자_월', '변경일자_일', '변경일자_년_월', '차종유형', 'Grade'
}

In [ ]:
dg4 = dg4.rename(columns=chc_col)
dg4.columns

# 코드 마지막